# 1. Data Exploration & Preparation

## 1.1 Imported Required Library & Datasets

In [1]:
import numpy as np
import sklearn
import pandas as pd
import datetime
from datetime import date
from sklearn import linear_model

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn


In [2]:
# Load up all the sheets in the CSV dataset
training_set = pd.read_excel('KPMG Data.xlsx', sheet_name='NewCustomerList')
training_target = training_set["Value"]

# Load list of customer to predict the value
CustomerDemographic =pd.read_excel('KPMG Data.xlsx', sheet_name='CustomerDemographic')
CustomerAddress = pd.read_excel('KPMG Data.xlsx', sheet_name='CustomerAddress')
# Merge customer data with its address details
testing_set = pd.merge(CustomerDemographic, CustomerAddress, on='customer_id')

## 1.2 Data Exploration

In [3]:
print("Training set size is {} ".format(training_set.shape))
print("Testing set size is {}".format(testing_set.shape))

Training set size is (1000, 18) 
Testing set size is (3996, 18)


In [4]:
training_set.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125


In [5]:
training_set.describe()

,past_3_years_bike_related_purchases,tenure,postcode,property_valuation,Rank,Value
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,49.836000,11.388000,3019.227000,7.397000,498.819000,0.881714
std,27.796686,5.037145,848.895767,2.758804,288.810997,0.293525
min,0.000000,0.000000,2000.000000,1.000000,1.000000,0.340000
25%,26.750000,7.000000,2209.000000,6.000000,250.000000,0.649531
50%,51.000000,11.000000,2800.000000,8.000000,500.000000,0.860000
75%,72.000000,15.000000,3845.500000,9.000000,750.250000,1.075000
max,99.000000,22.000000,4879.000000,12.000000,1000.000000,1.718750


In [6]:
testing_set.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0,0 Holy Cross Court,4211,QLD,Australia,9
3,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,Curr,Duckhouse,Male,35,1966-09-16,NaN,Retail,High Net Worth,N,ðµ ð ð ð,Yes,13.0,9 Oakridge Court,3216,VIC,Australia,9


## 1.3 Data Pre-processing

### 1.3.1 Feature Selection

In [7]:
training_set.drop(['first_name', 'last_name', 'deceased_indicator', 'address', 'postcode', 'country', 'Rank', 'Value'], axis=1, inplace=True)
testing_set.drop(['customer_id', 'first_name', 'last_name', 'deceased_indicator', 'default','address', 'postcode', 'country'], axis=1, inplace=True)

In [8]:
training_set.head()

,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,owns_car,tenure,state,property_valuation
0,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,Yes,14,QLD,6
1,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,No,16,NSW,11
2,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,No,10,VIC,5
3,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,Yes,5,QLD,1
4,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,No,19,NSW,9


In [9]:
testing_set.head()

,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,owns_car,tenure,state,property_valuation
0,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,Yes,11.0,New South Wales,10
1,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,Yes,16.0,New South Wales,10
2,Male,33,1961-10-03,NaN,IT,Mass Customer,No,7.0,QLD,9
3,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,Yes,8.0,New South Wales,4
4,Male,35,1966-09-16,NaN,Retail,High Net Worth,Yes,13.0,VIC,9


In [11]:
# Store size of training and testing set for reassembling
ntrain = training_set.shape[0]
ntest = testing_set.shape[0]

In [13]:
# Concatenate all data for data pre-processing
all_data = pd.concat((training_set, testing_set))

### 1.3.1 Handling Missing Values

In [15]:
print(all_data.isnull().sum())

gender                                   0
past_3_years_bike_related_purchases      0
DOB                                    104
job_title                              610
job_industry_category                  820
wealth_segment                           0
owns_car                                 0
tenure                                  87
state                                    0
property_valuation                       0
dtype: int64


In [16]:
# Handing categrocial misisng data
categorical_miss_data = ["job_title", "job_industry_category"]
all_data[categorical_miss_data] = all_data[categorical_miss_data].fillna("unknown")

In [21]:
# Handling numerical missing data
all_data["tenure"] = all_data["tenure"].fillna(all_data["tenure"].mean())

In [22]:
all_data["DOB"] = all_data["DOB"].fillna(all_data["DOB"].mean())

### 1.3.2 Abbrevate all state & Gender

In [27]:
Aus_state_abv = {
    'New South Wales':'NSW',
    'Victoria': 'VIC',
    'South Australia': 'SA',
    'Queensland': 'QLD',
    'NSW': 'NSW',
    'QLD': 'QLD',
    'VIC': 'VIC'
}

gender_abv = {
    'Male':'M',
    'Female':'F',
    'Femal': 'F',
    'F': 'F',
    'M': 'M'
}

all_data['state_abv'] = all_data['state'].map(Aus_state_abv)
all_data['gender_abv'] = all_data['gender'].map(gender_abv)


In [28]:
all_data.drop(['gender', 'state'], axis=1, inplace=True)

### 1.3.3 Converting DOB into age

In [29]:
age = ( (date.today() - all_data['DOB'].dt.date) / 365 ).dt.days
all_data.insert(3, "age", age)

In [30]:
all_data.drop("DOB", axis=1, inplace=True)

In [31]:
all_data.head()

,past_3_years_bike_related_purchases,job_title,age,job_industry_category,wealth_segment,owns_car,tenure,property_valuation,state_abv,gender_abv
0,86,General Manager,62,Manufacturing,Mass Customer,Yes,14.0,6,QLD,M
1,69,Structural Engineer,50,Property,Mass Customer,No,16.0,11,NSW,M
2,10,Senior Cost Accountant,45,Financial Services,Affluent Customer,No,10.0,5,VIC,F
3,64,Account Representative III,41,Manufacturing,Affluent Customer,Yes,5.0,1,QLD,F
4,34,Financial Analyst,54,Financial Services,Affluent Customer,No,19.0,9,NSW,F


### 1.3.3 Turn all data into numerical

In [34]:
from sklearn.preprocessing import LabelEncoder
cols=('job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'state_abv', 'gender_abv')

for c in cols:
  lbl = LabelEncoder()
  lbl.fit(list(all_data[c].values))
  all_data[c] = lbl.transform(list(all_data[c].values))

In [35]:
all_data.head()

,past_3_years_bike_related_purchases,job_title,age,job_industry_category,wealth_segment,owns_car,tenure,property_valuation,state_abv,gender_abv
0,86,76,62,5,2,1,14.0,6,1,1
1,69,174,50,6,2,0,16.0,11,0,1
2,10,147,45,2,0,0,10.0,5,2,0
3,64,4,41,5,0,1,5.0,1,1,0
4,34,73,54,2,0,0,19.0,9,0,0


In [37]:
all_data = pd.get_dummies(all_data)

# 2. Model Preparation & Training

### 2.1 Create training test set

In [38]:
train = all_data[:ntrain]
test = all_data[ntrain:]

### 2.2 Evaluate Model

In [62]:
Lassomodel = linear_model.LassoLars(alpha=.6)
Lassomodel.fit(train, training_target)

LassoLars(alpha=0.6, copy_X=True, eps=2.220446049250313e-16, fit_intercept=True,
          fit_path=True, max_iter=500, normalize=True, positive=False,
          precompute='auto', verbose=False)

In [63]:
result = Lassomodel.predict(test)

In [64]:
print(pd.DataFrame(result))
result = pd.DataFrame(result)

             0
0     0.881714
1     0.881714
2     0.881714
3     0.881714
4     0.881714
...        ...
3991  0.881714
3992  0.881714
3993  0.881714
3994  0.881714
3995  0.881714

[3996 rows x 1 columns]
